In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import os
import torch
from torch.utils.data import Dataset, DataLoader

from sklearn.feature_extraction import DictVectorizer
from sklearn.externals import joblib

from util import *

In [2]:
# %load_ext autoreload
# %autoreload 2

In [3]:
use_gpu = torch.cuda.is_available()
print(use_gpu)

False


In [4]:
data_path = '../data'
data_dict = joblib.load(os.path.join(data_path, 'data_dict.pkl'))

In [5]:
# Key = starting state, value = next state
transition_dict_train = dict(zip(data_dict['train']['state_id'], data_dict['train']['next_state_id']))
transition_dict_val = dict(zip(data_dict['val']['state_id'], data_dict['val']['next_state_id']))

In [6]:
config = {
              'state_dim' : data_dict['train']['X'].shape[1],
              'action_dim' : 25,
              'gamma' : 0.9,
              'batch_size' : 512,
              'lr' : 1e-4,
              'num_epochs' : 100,
              'hidden_dim' : 24,
              'num_hidden' : 5,
              'drop_prob' : 0.0,
              'target_update': 10,
              'option' : 'linear',
              'use_scheduler' : False
        }

In [7]:
print(data_dict['train']['X'].shape)
print(data_dict['val']['X'].shape)


(153569, 46)
(19540, 46)


In [8]:
# Create a Dataset
train_dataset = RL_Dataset(data_dict['train']['X'], 
                           data_dict['train']['action'],
                           data_dict['train']['reward'],
                           transition_dict_train)

val_dataset = RL_Dataset(data_dict['val']['X'], 
                           data_dict['val']['action'],
                           data_dict['val']['reward'],
                           transition_dict_val)

In [ ]:
# Create a dataloader
train_loader = DataLoader(train_dataset, 
                        config['batch_size'],
                        shuffle = True,
                        num_workers = 32
#                         collate_fn = my_collate
                         )

val_loader = DataLoader(val_dataset, 
                         config['batch_size'],
                         shuffle = True,
                         num_workers = 32
#                         collate_fn = my_collate
                         )

In [ ]:
loaders = {'train' : train_loader,
           'val' : val_loader
          }

dset_sizes = {'train' : len(train_dataset),
              'val' : len(val_dataset)
             }


model = dueling_net(D_in = config['state_dim'], 
                    H = config['hidden_dim'], 
                    D_out = config['action_dim'],
                    drop_prob = config['drop_prob'],
                    num_hidden = config['num_hidden'],
                    option = config['option']
                   )

target_model = dueling_net(D_in = config['state_dim'], 
                            H = config['hidden_dim'], 
                            D_out = config['action_dim'],
                            drop_prob = config['drop_prob'],
                            num_hidden = config['num_hidden'],
                            option = config['option']
                          )

optimizer = optim.Adam([{'params': model.parameters()}], 
                        lr = config['lr'])

if config['use_scheduler']:
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', verbose = True)
else:
    scheduler = None

def weights_init(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform(m.weight.data).float()

model.apply(weights_init)
target_model.apply(weights_init)

if use_gpu:
    model = model.cuda()
    target_model.cuda()

criterion = torch.nn.SmoothL1Loss(size_average = False)

performance_dict, best_model, best_loss, time_elapsed = train_model_double(model = model, 
                                                                            target_model = target_model,
                                                                            loaders = loaders, 
                                                                            dset_sizes = dset_sizes, 
                                                                            config = config, 
                                                                            criterion = criterion,
                                                                            optimizer = optimizer,
                                                                            scheduler = scheduler,
                                                                            use_gpu = use_gpu)
# loss_dict[name] = best_loss

----------
Epoch 0/99
----------
Updating Target Model
train Loss: 1.423674, Best Value: 0.635803, Empirical Value: -0.194977
val Loss: 1.591676, Best Value: 0.607954, Empirical Value: -0.159891
Best Value updated
----------
Epoch 1/99
----------
train Loss: 1.444079, Best Value: 0.804370, Empirical Value: 0.150163
val Loss: 1.328656, Best Value: 0.925831, Empirical Value: 0.350447
Best Value updated
----------
Epoch 2/99
----------
train Loss: 1.339452, Best Value: 0.961703, Empirical Value: 0.447062
val Loss: 1.297089, Best Value: 0.983865, Empirical Value: 0.503225
Best Value updated
----------
Epoch 3/99
----------
train Loss: 1.324837, Best Value: 0.963877, Empirical Value: 0.522545
val Loss: 1.290932, Best Value: 0.935136, Empirical Value: 0.521515
----------
Epoch 4/99
----------
train Loss: 1.320989, Best Value: 0.919744, Empirical Value: 0.527066
val Loss: 1.288822, Best Value: 0.898995, Empirical Value: 0.520884
----------
Epoch 5/99
----------
train Loss: 1.319315, Best Valu